## wgMLST for mbovis

* http://www3.ridom.de/seqsphere/cgmlst/
* https://www.cgmlst.org/ncs/schema/741110/
* https://jcm.asm.org/content/52/7/2479
* https://www.biorxiv.org/content/10.1101/172858v1.full
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3980634/

## steps

* get MLST gene list
* assembly
* annotate
* assign allele numbers?

In [40]:
import sys,os,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 150)
import pylab as plt
from Bio import SeqIO
from io import StringIO
import snipgenie as snpg
import pathogenie as pg

In [ ]:
path = '/storage/btbgenie/mbovis_ireland/Wicklow/Fastqs_07-01-18/'
files = glob.glob(os.path.join(path, '*.gz'))
samp = app.get_samples(files)
pairs = pd.pivot_table(samp,index=['sample'],columns=['pair'],values='filename',aggfunc='first').reset_index()
print (pairs)

## assembly

In [ ]:
def spades(file1, file2, outpath,threads=4):
    """Run spades"""
    
    cmd = 'spades -t %s --pe1-1 %s --pe1-2 %s --careful -o %s' %(threads,file1,file2,outpath)
    print (cmd)
    if not os.path.exists(out): 
        subprocess.check_output(cmd, shell=True)
    return

path = '/storage/btbgenie/assembly/'

for i,r in pairs[:1].iterrows():
    print (r[1])
    out = os.path.join(path,r['sample'])
    spades(r[1], r[2], out,threads=10)

## get consensus sequence from alignment instead of assembly?

In [51]:
vcf_file='/storage/btbgenie/all_ireland_results/filtered.vcf.gz'
sample = '1034'
cmd='bcftools index %s' %vcf_file
#subprocess.check_output(cmd, shell=True)
cmd='cat {r} | bcftools consensus -s {s} {v} > consensus.fa'.format(r=app.mbovis_genome,v=vcf_file,s=sample)
print (cmd)
#subprocess.check_output(cmd, shell=True)

cat /home/damien/.config/snipgenie/genome/Mbovis_AF212297.fa | bcftools consensus -s 1034 /storage/btbgenie/all_ireland_results/filtered.vcf.gz > consensus.fa


## annotate

need to use known proteins with Mb tags

In [ ]:
#seqfile='/storage/btbgenie/assembly/17/scaffolds.fasta'
seqfile='consensus.fa'
featdf,recs = pg.run_annotation(seqfile,
                                threads=10, kingdom='bacteria', trusted='Mbovis_AF212297_protein.fasta')
fastafile='/storage/btbgenie/annotation/17.fa'
df=pg.tools.records_to_dataframe(recs)
pg.tools.dataframe_to_fasta(df,outfile=fastafile)

In [68]:
## create a master table of alleles and add each new isolate to it?


1

In [84]:
mlst=pd.read_csv('mlst_scheme.csv')
mlst=mlst[mlst['Ridom Usage']=='MLST+']

In [86]:
mlst

,Target,Ridom Usage,Ridom Exclusion Reason,location,gene,product,protein_id,note
13,Rv0014c,MLST+,-,15590..17470,pknB,Transmembrane serine/threonine-protein kinase B PknB (protein kinase B) (STPK B),NP_214528.1,"Rv0014c, (MTCY10H4.14c), len: 626 aa. PknB,transmembrane serine/threonine-protein kinase (see citations below). Contains PS00107 Protein kinases A..."
14,Rv0015c,MLST+,-,17467..18762,pknA,Transmembrane serine/threonine-protein kinase A PknA (protein kinase A) (STPK A),NP_214529.1,"Rv0015c, (MTCY10H4.15c), len: 431 aa. PknA,transmembrane serine/threonine-protein kinase,magnesium/manganese dependent (see citations below). Cont..."
15,Rv0016c,MLST+,-,18759..20234,pbpA,Probable penicillin-binding protein PbpA,NP_214530.1,"Rv0016c, (MTCY10H4.16c), len: 491 aa. Probable pbpA,penicillin-binding protein, similar to many."
16,Rv0017c,MLST+,-,20231..21640,rodA,Probable cell division protein RodA,NP_214531.1,"Rv0017c, (MTCY10H4.17c), len: 469 aa. Probable rodA (alternate gene name: ftsW), cell division protein,integral membrane protein. Belongs to the F..."
17,Rv0018c,MLST+,-,21637..23181,pstP,Phosphoserine/threonine phosphatase PstP,NP_214532.1,"Rv0018c, (MTCY10H4.18c), len: 514 aa. PstP,phosphoserine/threonine phosphatase. Experimental studies have shown that PstP specifically dephosporyl..."
...,...,...,...,...,...,...,...,...
3999,Rv3906c,MLST+,-,4391097..4391606,NaN,hypothetical protein,NP_218423.1,"Rv3906c, (MTCY15F10.05), len: 169 aa. Conserved hypothetical protein, strongly related to Q50578|AT9S (sod related in Escherichia coli) from Mycob..."
4000,Rv3907c,MLST+,-,4391631..4393073,pcnA,Probable poly(A) polymerase PcnA (polynucleotide adenylyltransferase) (NTP polymerase) (RNA adenylating enzyme) (poly(A) polymerase),YP_178026.1,"Rv3907c, (MTCY15F10.04), len: 480 aa. Probable pcnA,polynucleotide polymerase, equivalent to Q9CCY1|PCNA|ML2697 PCNA protein from Mycobacterium le..."
4001,Rv3908,MLST+,-,4393449..4394195,mutT4,Possible mutator protein MutT4,NP_218425.1,"Rv3908, (MTCY15F10.03c), len: 248 aa. Possible mutT4, mutator protein, equivalent to Q50195|ML2698|L222-ORF6 hypothetical protein from Mycobacteri..."
4002,Rv3909,MLST+,-,4394192..4396600,NaN,hypothetical protein,NP_218426.1,"Rv3909, (MTCY15F10.02c), len: 802 aa. Conserved protein, equivalent to Q9CCY0|ML2699 putative secreted protein from Mycobacterium leprae (797 aa) ..."
